In [1]:
from dotenv import load_dotenv
import os
from astrapy import DataAPIClient
from langchain_community.document_loaders import WebBaseLoader

load_dotenv()


USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')


Initializing LLM

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.2,
    max_retries=2,

)

In [4]:
llm.invoke("tell me about a flag")

AIMessage(content='Let me tell you about the flag of Japan!\n\nThe flag of Japan, also known as the Nisshoki, is a white rectangular flag with a large red disk (representing the sun) in the center. The flag is also known as the "Rising Sun" flag, and it has been the national flag of Japan since 1870.\n\nThe design of the flag is based on the ancient Japanese symbol of the sun, which was used by the Japanese Empire during the 16th century. The red disk represents the sun, while the white background represents the purity and innocence of the Japanese people.\n\nThe flag has undergone several changes throughout its history. In the early 20th century, the flag was modified to include a blue triangle at the hoist side, but this design was abandoned in 1945 after Japan\'s defeat in World War II. The current design, with the red disk on a white background, has been used since 1870.\n\nThe flag is an important symbol of Japanese identity and is often displayed at government buildings, schools,

Load the Job Content from the website

In [75]:
loader =WebBaseLoader("https://jobs.tkxel.com/jobs/Careers/524295000024517645/Senior-Software-Engineer-Python-Ben?source=CareerSite")
page_data=loader.load()
page_data

[Document(metadata={'source': 'https://jobs.tkxel.com/jobs/Careers/524295000024517645/Senior-Software-Engineer-Python-Ben?source=CareerSite', 'title': 'Tkxel - Senior Software Engineer Python/Ben in Lahore', 'description': 'Tkxel - Develop, and maintain efficient, reusable code using Python. - Must have worked on different frameworks like Django, Flask etc. - Strong under', 'language': 'en'}, page_content='\nTkxel - Senior Software Engineer Python/Ben in Lahore\n')]

In [78]:
document_string = str(page_data)

Prompt template for extracting information

In [91]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
       """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in Just JSON format 
        'Role':
        'skills':
        'description':
        Wrap one job opening in a single object.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

Chaining the above process

In [98]:
chain_extract = prompt | llm 
res = chain_extract.invoke(input={'page_data':page_data})
# type(res.content)

In [104]:
response=res.content

Parsing the output

In [106]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(response)
json_res

{'Role': 'Senior Software Engineer Python/Ben',
 'Skills': ['Python', 'Django', 'Flask'],
 'Description': 'Develop, and maintain efficient, reusable code using Python. Must have worked on different frameworks like Django, Flask etc. Strong understanding of Python and its ecosystem.'}

In [108]:
type(json_res)

dict

In [109]:
job=json_res

In [107]:
# type(json_res)
json_res['Skills']

['Python', 'Django', 'Flask']

In [43]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


# AstraDB

In [44]:
from langchain_core.embeddings import FakeEmbeddings

embeddings = FakeEmbeddings(size=1536)

In [45]:
ASTRA_DB_API_ENDPOINT="<ur endpoint>"
ASTRA_DB_APPLICATION_TOKEN="<ur token>"
ASTRA_DB_NAMESPACE="<name>"

In [118]:
from langchain_astradb import AstraDBVectorStore

vector_store = AstraDBVectorStore(
    collection_name="<collection name>",
    embedding=embeddings,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_NAMESPACE,
)

In [46]:
from uuid import uuid4

from langchain_core.documents import Document

In [119]:
# Prepare documents list
documents = [Document(page_content=row["Techstack"], metadata={"links": row["Links"]}) for _, row in df.iterrows()]

# Prepare ids list
ids = [str(uuid4()) for _ in range(len(documents))]

# Add documents to the vector store
vector_store.add_documents(
    documents=documents,
    ids=ids
)

['b050fb34-ee21-41f9-8498-62564b10f61d',
 '4ca7ca80-2451-4871-9fa3-22935fe1e0ef',
 '7a985179-8f17-40cf-844d-fc6dad635d09',
 '11077f02-fdc4-4656-8d10-2825f54a9a73',
 '52400516-9c7f-4bb1-ad46-24223e00aac6',
 'c92449a2-2167-4dc9-9e19-39dc6b719d52',
 'b26b43bc-6be5-4a39-b176-f6080829772d',
 '432ce81f-3f34-4c52-8a1f-b33d6c3a5061',
 '8693c58f-e012-4d9b-a9fe-8a1110af9f40',
 '7c1b2814-c722-44a6-9108-2f10fb7e731a',
 'a9e556a6-68e1-41e3-b28c-f654eb785431',
 '1d0ddfce-8cbd-4b1f-8877-63a01de96224',
 '2556542d-cf40-4d40-81e4-ae7871e6e655',
 '46f6a1f9-f7f7-4846-ae39-f68ac449b273',
 'd926db9a-3c6f-468a-b52e-38acb3f13259',
 'bfe6da58-1e69-486f-ae87-167814b9cc41',
 'd8af8bf1-d14c-4708-9796-410fdb886369',
 '9e91f0e1-69eb-4434-9742-bfe661762eab',
 'ece91db4-c5eb-4fce-adba-339b63d9aead',
 '20bd6f3d-b0a3-4bd4-9945-41954ecd2844']

In [110]:
job=json_res


In [120]:
showw=embeddings.aembed_query(job['Skills'])

In [121]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.5},
)
links=retriever.invoke(showw, filter={"source": "Links"})
links

[Document(metadata={'links': 'https://example.com/react-portfolio'}, page_content='React, Node.js, MongoDB'),
 Document(metadata={'links': 'https://example.com/xamarin-portfolio'}, page_content='Cross-platform, Xamarin, Azure'),
 Document(metadata={'links': 'https://example.com/devops-portfolio'}, page_content='DevOps, Jenkins, Docker')]

# ELASTIC SEARCH

In [128]:
from langchain_elasticsearch import ElasticsearchStore
elastic_vector_search = ElasticsearchStore(
    es_cloud_id="<ur id>",
    index_name="test_index",
    embedding=embeddings,
    es_user="elastic",
    es_password="<ur password>",
)

# elastic_vector_search = ElasticsearchStore(
#     es_url="http://localhost:8200",
#     es_api_key="",
#     index_name="",
#     embedding=embeddings,
#     es_user="elastic",
#     es_password="",
# )

In [129]:
documents = [Document(page_content=row["Techstack"], metadata={"links": row["Links"]}) for _, row in df.iterrows()]

# Prepare ids list
ids = [str(uuid4()) for _ in range(len(documents))]

# Add documents to the vector store
elastic_vector_search.add_documents(
    documents=documents,
    ids=ids
)

['804c12ce-14a6-46cf-8331-0cd152756851',
 '8b96a6a6-3def-4fca-8031-6c790e4b6835',
 'a9abe10d-74ab-4b66-9080-ad8d198be6e0',
 '588afc6d-d4c4-4503-944e-c331945a126c',
 'bdbf2b18-90b2-48b2-b79c-3a0f1fc0cf0e',
 'bec1c312-8ccf-4766-bad2-eedeb50aa687',
 '6c9a0158-df43-4afa-959e-0b63a78b0bff',
 '5763fc7c-c991-43ef-9f07-c8b407c994e2',
 '97b5ecca-d0d7-47c5-b506-adfcd069b11e',
 '137039ba-3288-409c-aba7-7ea1b6ee8e1e',
 'ca2c6485-670c-400a-b3fa-047862f53b95',
 '9de5ebdd-b446-4123-bf23-1c8302337c8e',
 '13ed8786-94fa-400e-894f-5e80ee0e70dd',
 'ee872168-f1b4-4e55-a08c-d920362a4cbe',
 '556f93fd-88d8-4990-8044-09655e0624f0',
 '307b9e1f-f288-41a6-b8a6-ec2a155cfb0a',
 '9853a0b3-a6d1-4c52-b1f6-ac7598384d6c',
 '1ed2cfce-38c1-41df-a3d8-50f74a007c32',
 'a1906f0d-f20c-4ee7-8d04-3ca0c766b7d5',
 '697a4ac6-6d38-4a6c-adf3-6da61953ffc5']

In [131]:
job['Skills']

['Python', 'Django', 'Flask']

In [142]:
type(job)

dict

# COLD EMAIL GENERATION USING ELASTIC SEARCH

In [156]:
query=embeddings.embed_query(job['Skills'])

In [160]:
retriever = elastic_vector_search.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)
links=retriever.invoke(job['Skills'])
links

[Document(metadata={'links': 'https://example.com/java-portfolio'}, page_content='Java, Spring Boot, Oracle'),
 Document(metadata={'links': 'https://example.com/ml-python-portfolio'}, page_content='Machine Learning, Python, TensorFlow'),
 Document(metadata={'links': 'https://example.com/ios-ar-portfolio'}, page_content='iOS, Swift, ARKit'),
 Document(metadata={'links': 'https://example.com/kotlin-android-portfolio'}, page_content='Kotlin, Android, Firebase')]

In [163]:
prompt_email = ChatPromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are XYZ, a business development executive at CompanyAlasks. CompanyAlasks is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of CompanyAlasks 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase all portfolios: {link_list} in structured format.
        Remember you are XYZ, BDE at CompanyAlasks. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Python Development Solutions for Your Business

Dear [Recipient],

I came across your job posting for a Senior Software Engineer with expertise in Python and its frameworks, and I wanted to introduce you to CompanyAlasks, a leading AI & Software Consulting company. Our team of experts has a proven track record of delivering tailored solutions that drive business growth, scalability, and efficiency.

At CompanyAlasks, we have a strong background in developing efficient, reusable code using Python, with experience in frameworks like Django and Flask. Our team is well-versed in the Python ecosystem and has a deep understanding of its capabilities. We would be delighted to leverage our expertise to help you achieve your project goals.

To give you a glimpse into our capabilities, I'd like to share some of our recent projects:

* **Machine Learning and Python**: We have successfully developed machine learning models using Python and TensorFlow, helping clients make data-driv

# COLD EMAIL GENERATION USING ASTRADB

In [143]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)
retriever.invoke(job['Skills'])

[Document(metadata={'links': 'https://example.com/ml-python-portfolio'}, page_content='Machine Learning, Python, TensorFlow'),
 Document(metadata={'links': 'https://example.com/java-portfolio'}, page_content='Java, Spring Boot, Oracle'),
 Document(metadata={'links': 'https://example.com/kotlin-backend-portfolio'}, page_content='Backend, Kotlin, Spring Boot'),
 Document(metadata={'links': 'https://example.com/devops-portfolio'}, page_content='DevOps, Jenkins, Docker')]

In [148]:
prompt_email = ChatPromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are XYZ, a business development executive at CompanyAlasks. CompanyAlasks is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of CompanyAlasks 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase all portfolios: {link_list} in a structured format.
        Remember you are XYZ, BDE at CompanyAlasks. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Python Development Solutions for Your Business

Dear [Recipient],

I came across your job posting for a Senior Software Engineer with expertise in Python and its frameworks, and I wanted to introduce you to CompanyAlasks, a leading AI & Software Consulting company. Our team of experts has a proven track record of delivering tailored solutions that drive business growth, scalability, and efficiency.

At CompanyAlasks, we have a strong background in developing efficient, reusable code using Python, with experience in frameworks like Django and Flask. Our team is well-versed in the Python ecosystem and has a deep understanding of its capabilities. We would be delighted to leverage our expertise to help you achieve your project goals.

I'd like to highlight some of our notable projects that demonstrate our capabilities:

* [Document(metadata={'links': 'https://example.com/react-portfolio'}, page_content='React, Node.js, MongoDB')]: We have successfully developed scalable an